In [1]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [3]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Cell_Phones_and_Accessories_5.json.gz')

In [5]:
df = df.loc[:,['reviewText','overall']]

In [7]:
import numpy as np
df['sentiment'] = np.where(df['overall'] >= 4, 1, 0)

In [8]:
df = df.drop(['overall'],axis=1)

In [10]:
df.columns = ['review','sentiment']

In [11]:
df.to_csv('reviews_Cell_Phones_and_Accessories.csv', index=False)

### 데이터 전처리

In [13]:
import pandas as pd
import numpy as np

In [14]:
nsmc = pd.read_csv('reviews_Cell_Phones_and_Accessories.csv')

In [15]:
nsmc = nsmc.dropna()

In [16]:
nsmc.shape

(194340, 2)

In [17]:
with open('nsmc.txt', 'w', encoding='utf8') as f:  
    f.write('\n'.join(nsmc['review']))  

In [18]:
from sentencepiece import SentencePieceTrainer
SentencePieceTrainer.Train('--input=nsmc.txt --model_prefix=nsmc --vocab_size=3000')  # 총 어휘수 3000개

True

In [19]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("nsmc.model")

True

In [22]:
from sklearn.feature_extraction.text import CountVectorizer 

In [23]:
cv = CountVectorizer(lowercase=False, tokenizer=sp.encode_as_pieces)  

In [24]:
tdm = cv.fit_transform(nsmc['review'])

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
x = tdm
y = nsmc['sentiment']

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [28]:
import joblib

In [29]:
cv.tokenizer = None

In [30]:
joblib.dump((cv,x_train,x_test,y_train,y_test), 'nsmc.pkl')

['nsmc.pkl']